<a href="https://colab.research.google.com/github/omarsayed7/RSNA-Pneumonia-Detection/blob/master/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data download

In [1]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset

Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1648 (delta 8), reused 26 (delta 7), pack-reused 1616
Receiving objects: 100% (1648/1648), 221.06 MiB | 35.68 MiB/s, done.
Resolving deltas: 100% (782/782), done.
Checking out files: 100% (356/356), done.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cp -r drive/My\ Drive/dense_2classes_25epoch.h5 ./

In [0]:
%cp -r drive/My\ Drive/normal ./

In [2]:
import os, random, cv2, shutil
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Dropout,Flatten, AveragePooling2D, GlobalAveragePooling2D, Concatenate, Conv2D
import keras.optimizers as optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import plot_model

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
keras.__version__

'2.3.1'

# Overview

In [0]:
metadata = pd.read_csv("covid-chestxray-dataset/metadata.csv")

In [7]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               360 non-null    int64  
 1   offset                  264 non-null    float64
 2   sex                     317 non-null    object 
 3   age                     306 non-null    float64
 4   finding                 360 non-null    object 
 5   survival                109 non-null    object 
 6   intubated               72 non-null     object 
 7   intubation_present      63 non-null     object 
 8   went_icu                27 non-null     object 
 9   needed_supplemental_O2  6 non-null      object 
 10  extubated               16 non-null     object 
 11  temperature             29 non-null     float64
 12  pO2_saturation          38 non-null     float64
 13  leukocyte_count         9 non-null      float64
 14  neutrophil_count        2 non-null      fl

# Data preperation 

In [0]:
data_info = metadata.dropna(axis=1)

In [0]:
data_info = data_info.drop(['url'],axis=1)

In [0]:
data = data_info.drop(data_info[data_info['modality']=='CT'].index, axis=0)

In [0]:
final_data = data_info[data_info['view']=='PA']

In [0]:
final_data = final_data[final_data['finding']=='COVID-19']

In [13]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 0 to 358
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patientid  145 non-null    int64 
 1   finding    145 non-null    object
 2   view       145 non-null    object
 3   modality   145 non-null    object
 4   folder     145 non-null    object
 5   filename   145 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.9+ KB




---


**I have considered the COVID-19 PA view X-ray scans for my analysis**


---







In [0]:
positive_covid =final_data.drop(['view'],axis=1)
positive_covid =positive_covid.drop(['modality'],axis=1)
positive_covid =positive_covid.drop(['finding'],axis=1)
positive_covid =positive_covid.drop(['folder'],axis=1)

In [15]:
positive_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145 entries, 0 to 358
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   patientid  145 non-null    int64 
 1   filename   145 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.4+ KB


In [16]:
positive_covid.head()

,patientid,filename
0,2,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1,2,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2,2,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3,2,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4,4,nejmc2001573_f1a.jpeg


In [0]:
listtt = positive_covid['filename']

In [18]:
len(positive_covid)

145

In [19]:
listtt

0      auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...
1      auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...
2      auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...
3      auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...
4                                  nejmc2001573_f1a.jpeg
                             ...                        
352                                     extubation-8.jpg
353                                    extubation-13.jpg
356                                        16708_1_1.jpg
357                                        16708_4_1.jpg
358                                        16708_5_1.jpg
Name: filename, Length: 145, dtype: object

And now we are going to use utility function to parse COVID-19 images into training forlder 

In [0]:
def parse_covid(metadata, img_dir, target_dir='covid'):
  print("Start parsing the images from {} directory".format(img_dir))
  img_files= metadata['filename']
  if not os.path.exists(target_dir):
    os.mkdir(target_dir)
  for img, idx in zip(img_files, range(len(metadata))):
    src = os.path.join(img_dir,img)
    dst = os.path.join(target_dir,str(idx)+'.png')
    shutil.copyfile(src, dst)
  print('Done copying {0} images from {1} directory'.format(len(metadata), img_dir))

In [21]:
parse_covid(positive_covid, 'covid-chestxray-dataset/images')

Start parsing the images from covid-chestxray-dataset/images directory
Done copying 145 images from covid-chestxray-dataset/images directory


**I will sample 120 images of normal patients from RSNA Pneumonia dataset**

So.. 

`Normal`: 120 images
`COVID-19`: 142 images 

# Data preprocessing

In [0]:
class Train_val_split:
  def __init__(self, split_size=0.15):
    self.split_size = split_size
    self.train_dir = 'train'
    self.val_dir = 'validation'
    if not os.path.exists(self.train_dir):
      os.mkdir(self.train_dir)
    if not os.path.exists(self.val_dir):
      os.mkdir(self.val_dir)

  def split_class(self,target_class):
    if not os.path.exists(self.train_dir+'/'+target_class):
      os.mkdir(self.train_dir+'/'+target_class)
    if not os.path.exists(self.val_dir+'/'+target_class):
      os.mkdir(self.val_dir+'/'+target_class)
    
    data_indices = list(range(len(os.listdir(target_class))))
    random.shuffle(data_indices) 
    split_size = (1-self.split_size)*len(data_indices)
    training_idx = data_indices[:int(split_size)]
    validation_idx = data_indices[int(split_size):]

    for idx in training_idx:
      src = os.path.join(target_class, str(idx)+'.png')
      dst = os.path.join(self.train_dir+'/'+target_class,target_class+'_'+str(idx)+'.png')
      shutil.copyfile(src, dst)
    
    for idx in validation_idx:
      src = os.path.join(target_class, str(idx)+'.png')
      dst = os.path.join(self.val_dir+'/'+target_class,target_class+'_'+str(idx)+'.png')
      shutil.copyfile(src, dst)

In [0]:
split_data = Train_val_split(0.15)

In [0]:
split_data.split_class('covid')
split_data.split_class('normal')

In [25]:
len(os.listdir('train/normal')), len(os.listdir('train/covid'))

(102, 123)

In [26]:
len(os.listdir('validation/normal')), len(os.listdir('validation/covid'))

(18, 22)

# Data loaders

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'train',
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        'validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 225 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


# Classification Model

In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',
                        input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',
              optimizer= optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 128)      

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/25
50/50 [==============================] - 207s 4s/step - loss: 0.5089 - acc: 0.7636 - val_loss: 0.6287 - val_acc: 0.7500
Epoch 2/25
50/50 [==============================] - 206s 4s/step - loss: 0.4298 - acc: 0.7914 - val_loss: 0.6937 - val_acc: 0.6750
Epoch 3/25
50/50 [==============================] - 203s 4s/step - loss: 0.4648 - acc: 0.7716 - val_loss: 0.5062 - val_acc: 0.7500
Epoch 4/25
50/50 [==============================] - 210s 4s/step - loss: 0.4393 - acc: 0.7850 - val_loss: 0.2756 - val_acc: 0.8000
Epoch 5/25
50/50 [==============================] - 209s 4s/step - loss: 0.4848 - acc: 0.7942 - val_loss: 0.2915 - val_acc: 0.7500
Epoch 6/25
50/50 [==============================] - 200s 4s/step - loss: 0.4259 - acc: 0.8012 - val_loss: 0.2803 - val_acc: 0.8000
Epoch 7/25
50/50 [==============================] - 204s 4s/step - loss: 0.3767 - acc: 0.8062 - val_loss: 0.5435 - val_acc: 0.7250
Epoch 8/25
50/50 [==============================] - 214s 4s/step - loss: 0.3832 - a

## Transfer Learning

In [0]:
from keras.models import load_model
model_transfer= load_model("dense_2classes_25epoch.h5")

In [0]:
model_2 = Model(inputs=model_transfer.input, outputs=model_transfer.output)

In [0]:
for layer in model_transfer.layers[:-2]:
  layer.trainable = False

In [10]:
model_2.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256, 256, 3)  0           input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           concatenate_1[0][0]              
____________________________________________________________________________________________

In [56]:
train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=10,
                zoom_range=[0.5,1.0],
                horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'train',
        # All images will be resized to 150x150
        target_size=(256, 256),
        color_mode= 'grayscale',
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        'validation',
        target_size=(256, 256),
        color_mode= 'grayscale',
        batch_size=32,
        class_mode='binary')

Found 225 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [57]:
model_2.compile(loss='binary_crossentropy',
              optimizer= optimizers.Adam(),
              metrics=['acc'])

history_2 = model_2.fit_generator(
      train_generator,
      steps_per_epoch=50,
      epochs=25,
      validation_data=validation_generator,
      validation_steps=50)

Epoch 1/25
50/50 [==============================] - 122s 2s/step - loss: 0.7718 - acc: 0.6195 - val_loss: 1.1042 - val_acc: 0.4500
Epoch 2/25
50/50 [==============================] - 75s 1s/step - loss: 0.4989 - acc: 0.7793 - val_loss: 3.1659 - val_acc: 0.4500
Epoch 3/25
50/50 [==============================] - 73s 1s/step - loss: 0.4254 - acc: 0.8344 - val_loss: 4.5305 - val_acc: 0.4500
Epoch 4/25
50/50 [==============================] - 77s 2s/step - loss: 0.3460 - acc: 0.8656 - val_loss: 1.4004 - val_acc: 0.4500
Epoch 5/25
50/50 [==============================] - 75s 1s/step - loss: 0.3646 - acc: 0.8840 - val_loss: 2.6029 - val_acc: 0.4500
Epoch 6/25
50/50 [==============================] - 74s 1s/step - loss: 0.2834 - acc: 0.9222 - val_loss: 7.5914 - val_acc: 0.4500
Epoch 7/25
50/50 [==============================] - 74s 1s/step - loss: 0.2621 - acc: 0.9255 - val_loss: 2.8896 - val_acc: 0.4500
Epoch 8/25
50/50 [==============================] - 78s 2s/step - loss: 0.2152 - acc: 0.9